In [2]:
# Creating a synthetic dataset (million-item dataset composed of random floating point values with mean 0 and variance 1)

In [3]:
import numpy as np
import random
import time
from sklearn.neighbors import NearestNeighbors

In [4]:
num_items = 100000
num_dimensions = 100
dataset = np.random.randn(num_items, num_dimensions)
dataset /= np.linalg.norm(dataset, axis=1).reshape(-1, 1)
randomIndex = random.randint(0, num_items)
query = dataset[randomIndex]

 Using timit command  to benchmark the time of a single operation

Brute Force

In [5]:
# Time the indexing for the brute force algorithm
%timeit NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(dataset)

100 loops, best of 3: 6.73 ms per loop


In [6]:
# Time the search for the brute force algorithm
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(dataset)
%timeit neighbors.kneighbors([query])

10 loops, best of 3: 22.1 ms per loop


Annoy 

Approximate Nearest Neighbors Oh Yeah 

(released by Spotify and is used in production to serve their music recommendations.)

In [7]:
pip install annoy

In [8]:
from annoy import AnnoyIndex
random_image_index = random.randint(0, num_items)
annoy_index = AnnoyIndex(
    num_dimensions)  # Length of item vector that will be indexed
for i in range(num_items):
    annoy_index.add_item(i, dataset[i])
annoy_index.build(40)  #40 trees
#Time the search for one image for Annoy
%timeit annoy_index.get_nns_by_vector(query, 5, include_distances=True )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  after removing the cwd from sys.path.


10000 loops, best of 3: 31.3 µs per loop


That's way faster when compared to brute force! Even for this million item dataset, this can serve almost 28000 requests on a single CPU core. Considering most CPUs have multiple cores, it should be able to handle more than 100,000 requests on a single system. It also lets us share the same index in memory between multiple processes.

k-d Tree

In [9]:
# Time the indexing for the k-d tree algorithm
%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(dataset)

1 loop, best of 3: 1.49 s per loop


In [10]:
# Time the search for the Ball Tree algorithm
neighbors = NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(dataset)
%timeit neighbors.kneighbors([query])

10 loops, best of 3: 34.9 ms per loop


Ball Tree

In [11]:
# Time the indexing for the Ball Tree algorithm
%timeit NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(dataset)

1 loop, best of 3: 1.09 s per loop


In [12]:
# Time the search for the Ball Tree algorithm
neighbors = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(dataset)
%timeit neighbors.kneighbors([query])

10 loops, best of 3: 29.9 ms per loop


NMS Lib

In [13]:
pip install nmslib

In [14]:
import nmslib
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(dataset)
index.createIndex({'post': 2}, print_progress=True)

In [15]:
# query for the nearest neighbors of the first datapoint
%timeit index.knnQuery(query, k=5)
ids, distances = index.knnQuery(query, k=5)

The slowest run took 16.95 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 30.5 µs per loop


In [16]:
# Get all nearest neighbors for all the datapoint
%timeit index.knnQueryBatch(dataset, k=5, num_threads=16)
neighbors = index.knnQueryBatch(dataset, k=5, num_threads=16)

1 loop, best of 3: 6.66 s per loop


Falconn

In [19]:
pip install falconn

     |████████████████████████████████| 1.4MB 9.0MB/s 
  Created wheel for falconn: filename=FALCONN-1.3.1-cp36-cp36m-linux_x86_64.whl size=10581238 sha256=911051e84c65feb2ad4fda5b3c3a7c74d0277e8e2bc8cb7e7404f17ad72a4b22
  Stored in directory: /root/.cache/pip/wheels/bf/36/96/d5538901888620fc0343c1ed9d5f87fce00869e00c12056ef8
Successfully built falconn


In [20]:
import falconn
parameters = falconn.LSHConstructionParameters()
num_tables = 1
parameters.l = num_tables
parameters.dimension = num_dimensions
parameters.distance_function = falconn.DistanceFunction.EuclideanSquared
parameters.lsh_family = falconn.LSHFamily.CrossPolytope
parameters.num_rotations = 1
parameters.num_setup_threads = 1
parameters.storage_hash_table = falconn.StorageHashTable.BitPackedFlatHashTable
falconn.compute_number_of_hash_functions(16, parameters)

index = falconn.LSHIndex(parameters)
%time index.setup(dataset)

query_object = index.construct_query_object()
num_probes = 1
query_object.set_num_probes(num_probes)

%timeit query_object.find_k_nearest_neighbors(query, 5)

CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 108 ms
The slowest run took 157.20 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.74 µs per loop
